In [1]:
import pickle
import pandas as pd
import torch
import torch.nn as nn
import numpy as np

from utils import seed_everything, set_device, get_train_val_dataloaders, train
from model import FeedForwardRegressionNet

from torch.utils.data import TensorDataset
from torch.optim import Adam

In [2]:
n_jobs = 1000
results = []

for i in range(n_jobs):
    results.append(pickle.load(open(f"results/training_stats_{i}", "rb")))

In [3]:
df = pd.DataFrame(results).sort_values("Valid. Loss")

In [4]:
df[0:10]

,Training Loss,Valid. Loss,Valid. CI,Training Time,Validation Time,hidden_dims,learning_rate,embed_dim0,embed_dim1,epochs,job_id,weight_decay
462,6064.796328,5033.432192,724.901440,0:00:03,0:00:00,"[64, 128]",0.010,16,4,32,462,0.0030
680,6211.431389,5263.054191,721.012220,0:00:02,0:00:00,"[64, 128]",0.010,4,32,32,680,0.0003
753,5520.396385,5797.777871,993.686591,0:00:02,0:00:00,"[64, 128]",0.010,16,16,32,753,0.0010
600,6151.538141,5896.051965,1197.892402,0:00:02,0:00:00,"[128, 8]",0.010,4,16,32,600,0.0030
438,6514.384305,5931.405604,675.170082,0:00:02,0:00:00,"[128, 16]",0.010,4,4,32,438,0.0000
959,5630.662366,6357.435193,946.263469,0:00:01,0:00:00,"[128, 16]",0.010,16,16,32,959,0.0010
41,6477.138495,6382.839544,848.396084,0:00:03,0:00:00,"[128, 64]",0.010,16,32,32,41,0.0000
683,6739.437005,6529.928876,933.661296,0:00:02,0:00:00,"[128, 64]",0.010,16,32,32,683,0.0003
610,7261.850720,6559.227511,1000.420066,0:00:01,0:00:00,"[64, 128]",0.003,4,32,32,610,0.0003
209,6881.805862,6619.525103,926.978175,0:00:03,0:00:00,"[128, 16]",0.010,8,16,32,209,0.0003


In [9]:
best_hyperparameters = {
    "hidden_dims": [128, 64],
    "learning_rate": 0.01,
    "embed_dim0": 4,
    "embed_dim1": 8,
    "epochs": 32,
    "weight_decay" : 0.03
}

pickle.dump(best_hyperparameters, open("best_hyperparameters.pkl", "wb"))

In [10]:
hyperparameters = pickle.load(open("best_hyperparameters.pkl", "rb"))

# hyperparameters = {
#     "hidden_dims": [64, 128],
#     "learning_rate": 0.01,
#     "embed_dim0": 16,
#     "embed_dim1": 4,
#     "epochs": 32,
#     "weight_decay" : 0.003
# }

HIDDEN_DIMS = hyperparameters["hidden_dims"]
LEARNING_RATE = hyperparameters["learning_rate"]
EMBED_DIM0 = hyperparameters["embed_dim0"]
EMBED_DIM1 = hyperparameters["embed_dim1"]
EPOCHS = hyperparameters["epochs"]

batch_size = 64
val_size = 0.1

In [11]:
device = set_device()

X = pickle.load(open("preprocessed_data/X_train_tensor.pkl", "rb"))
y = pickle.load(open("preprocessed_data/y_train_tensor.pkl", "rb"))

dataset = TensorDataset(X, y)

train_dataloader, validation_dataloader = get_train_val_dataloaders(
    dataset, val_size, batch_size
)

No GPU available, using the CPU instead.
45,000 training samples
4,999 validation samples


In [12]:
input_dim = X.shape[1] - 2
vocab_size0 = len(np.unique(X[:, 0]))
vocab_size1 = len(np.unique(X[:, 1]))
output_dim = 1

model = FeedForwardRegressionNet(
    input_dim,
    vocab_size0,
    vocab_size1,
    EMBED_DIM0,
    EMBED_DIM1,
    HIDDEN_DIMS,
    output_dim,
)

In [13]:
optimizer = Adam(model.parameters(), lr=LEARNING_RATE)

criterion = nn.MSELoss()

seed_everything(42)

training_stats = train(
    model,
    EPOCHS,
    train_dataloader,
    validation_dataloader,
    criterion,
    optimizer,
    device=device,
    SSY=y.var() * len(y),
)


======== Epoch 1 / 32 ========
Training...
  Batch    40  of    704.    Elapsed: 0:00:00.
  Batch    80  of    704.    Elapsed: 0:00:00.
  Batch   120  of    704.    Elapsed: 0:00:00.
  Batch   160  of    704.    Elapsed: 0:00:00.
  Batch   200  of    704.    Elapsed: 0:00:00.
  Batch   240  of    704.    Elapsed: 0:00:00.
  Batch   280  of    704.    Elapsed: 0:00:01.
  Batch   320  of    704.    Elapsed: 0:00:01.
  Batch   360  of    704.    Elapsed: 0:00:01.
  Batch   400  of    704.    Elapsed: 0:00:01.
  Batch   440  of    704.    Elapsed: 0:00:01.
  Batch   480  of    704.    Elapsed: 0:00:01.
  Batch   520  of    704.    Elapsed: 0:00:01.
  Batch   560  of    704.    Elapsed: 0:00:01.
  Batch   600  of    704.    Elapsed: 0:00:01.
  Batch   640  of    704.    Elapsed: 0:00:01.
  Batch   680  of    704.    Elapsed: 0:00:01.

  Average training loss: 270278.99
  Training epoch took: 0:00:01

Running Validation...
  Validation Loss: 39414.66
  Validation CI: +/-3522.60
  Validatio

  Batch   600  of    704.    Elapsed: 0:00:01.
  Batch   640  of    704.    Elapsed: 0:00:01.
  Batch   680  of    704.    Elapsed: 0:00:01.

  Average training loss: 20578.37
  Training epoch took: 0:00:01

Running Validation...
  Validation Loss: 20020.49
  Validation CI: +/-2379.28
  Validation SSE: 100696800.00
  Validation took: 0:00:00
  Validation R^2: 0.9957311749458313

======== Epoch 9 / 32 ========
Training...
  Batch    40  of    704.    Elapsed: 0:00:00.
  Batch    80  of    704.    Elapsed: 0:00:00.
  Batch   120  of    704.    Elapsed: 0:00:00.
  Batch   160  of    704.    Elapsed: 0:00:00.
  Batch   200  of    704.    Elapsed: 0:00:00.
  Batch   240  of    704.    Elapsed: 0:00:00.
  Batch   280  of    704.    Elapsed: 0:00:01.
  Batch   320  of    704.    Elapsed: 0:00:01.
  Batch   360  of    704.    Elapsed: 0:00:01.
  Batch   400  of    704.    Elapsed: 0:00:01.
  Batch   440  of    704.    Elapsed: 0:00:01.
  Batch   480  of    704.    Elapsed: 0:00:01.
  Batch   5

  Batch   240  of    704.    Elapsed: 0:00:00.
  Batch   280  of    704.    Elapsed: 0:00:01.
  Batch   320  of    704.    Elapsed: 0:00:01.
  Batch   360  of    704.    Elapsed: 0:00:01.
  Batch   400  of    704.    Elapsed: 0:00:01.
  Batch   440  of    704.    Elapsed: 0:00:01.
  Batch   480  of    704.    Elapsed: 0:00:01.
  Batch   520  of    704.    Elapsed: 0:00:01.
  Batch   560  of    704.    Elapsed: 0:00:01.
  Batch   600  of    704.    Elapsed: 0:00:01.
  Batch   640  of    704.    Elapsed: 0:00:01.
  Batch   680  of    704.    Elapsed: 0:00:01.

  Average training loss: 9532.11
  Training epoch took: 0:00:01

Running Validation...
  Validation Loss: 8604.01
  Validation CI: +/-1333.53
  Validation SSE: 43400680.00
  Validation took: 0:00:00
  Validation R^2: 0.9981601238250732

======== Epoch 17 / 32 ========
Training...
  Batch    40  of    704.    Elapsed: 0:00:00.
  Batch    80  of    704.    Elapsed: 0:00:00.
  Batch   120  of    704.    Elapsed: 0:00:00.
  Batch   160

  Validation Loss: 5808.17
  Validation CI: +/-1087.71
  Validation SSE: 28818382.00
  Validation took: 0:00:00
  Validation R^2: 0.9987782835960388

======== Epoch 24 / 32 ========
Training...
  Batch    40  of    704.    Elapsed: 0:00:00.
  Batch    80  of    704.    Elapsed: 0:00:00.
  Batch   120  of    704.    Elapsed: 0:00:00.
  Batch   160  of    704.    Elapsed: 0:00:00.
  Batch   200  of    704.    Elapsed: 0:00:00.
  Batch   240  of    704.    Elapsed: 0:00:00.
  Batch   280  of    704.    Elapsed: 0:00:00.
  Batch   320  of    704.    Elapsed: 0:00:01.
  Batch   360  of    704.    Elapsed: 0:00:01.
  Batch   400  of    704.    Elapsed: 0:00:01.
  Batch   440  of    704.    Elapsed: 0:00:01.
  Batch   480  of    704.    Elapsed: 0:00:01.
  Batch   520  of    704.    Elapsed: 0:00:01.
  Batch   560  of    704.    Elapsed: 0:00:01.
  Batch   600  of    704.    Elapsed: 0:00:01.
  Batch   640  of    704.    Elapsed: 0:00:01.
  Batch   680  of    704.    Elapsed: 0:00:01.

  Aver

  Batch   480  of    704.    Elapsed: 0:00:01.
  Batch   520  of    704.    Elapsed: 0:00:01.
  Batch   560  of    704.    Elapsed: 0:00:01.
  Batch   600  of    704.    Elapsed: 0:00:01.
  Batch   640  of    704.    Elapsed: 0:00:01.
  Batch   680  of    704.    Elapsed: 0:00:01.

  Average training loss: 4616.64
  Training epoch took: 0:00:01

Running Validation...
  Validation Loss: 5936.66
  Validation CI: +/-984.34
  Validation SSE: 29229526.00
  Validation took: 0:00:00
  Validation R^2: 0.9987608790397644

======== Epoch 32 / 32 ========
Training...
  Batch    40  of    704.    Elapsed: 0:00:00.
  Batch    80  of    704.    Elapsed: 0:00:00.
  Batch   120  of    704.    Elapsed: 0:00:00.
  Batch   160  of    704.    Elapsed: 0:00:00.
  Batch   200  of    704.    Elapsed: 0:00:00.
  Batch   240  of    704.    Elapsed: 0:00:00.
  Batch   280  of    704.    Elapsed: 0:00:01.
  Batch   320  of    704.    Elapsed: 0:00:01.
  Batch   360  of    704.    Elapsed: 0:00:01.
  Batch   400 

In [14]:
torch.save(model.state_dict, "best_model.pt")